In [9]:
import pandas as pd
import re
import base64
from urllib.parse import unquote, urlparse

In [10]:
# 🔹 1. CSV 불러오기
df = pd.read_csv("../data/merged_output.csv")

C:\Users\gkswl\AppData\Local\Temp\ipykernel_22952\167397402.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/merged_output.csv")


In [11]:
# 🔸 사용할 컬럼명 정의
expected_cols = {
    'has_redirect_keyword': False,
    'has_base64_redirect': False,
    'has_fake_brand_domain': False,
    'is_short_url': False
}

# 🔸 분석 기준
redirect_keywords = ['redirect', 'url', 'next', 'target']
short_url_domains = ['bit.ly', 't.co', 'tinyurl.com', 'goo.gl', 'is.gd', 'ow.ly']
fake_brand_patterns = ['naver', 'kakao', 'google', 'apple', 'bank', 'login', 'secure']

# 🔸 분석 함수 정의
def analyze_url(url):
    if not isinstance(url, str):
        return pd.Series([False] * 4)

    parsed = urlparse(url)
    domain = parsed.netloc.lower()
    query = parsed.query.lower()

    has_redirect_keyword = any(kw in url.lower() for kw in redirect_keywords)

    decoded_query = unquote(query)
    looks_like_base64 = False
    try:
        decoded_value = base64.b64decode(decoded_query).decode('utf-8', errors='ignore')
        if 'http' in decoded_value:
            looks_like_base64 = True
    except:
        pass

    suspicious_domain = any(pattern in domain and not domain.endswith(f"{pattern}.com") for pattern in fake_brand_patterns)

    is_short_url = any(short in domain for short in short_url_domains)

    return pd.Series([
        has_redirect_keyword,
        looks_like_base64,
        suspicious_domain,
        is_short_url
    ])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617125 entries, 0 to 617124
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           617125 non-null  int64  
 1   url                  617125 non-null  object 
 2   type                 617125 non-null  object 
 3   original_url         617125 non-null  object 
 4   url_type             617125 non-null  int64  
 5   url_len              617125 non-null  int64  
 6   pri_domain           617125 non-null  object 
 7   letters_count        617125 non-null  int64  
 8   digits_count         617125 non-null  int64  
 9   special_chars_count  617125 non-null  int64  
 10  shortened            617125 non-null  int64  
 11  is_domain_in_url     617125 non-null  int64  
 12  secure_http          617125 non-null  int64  
 13  have_ip              617125 non-null  object 
 14  url_region           617125 non-null  object 
 15  root_domain      

In [15]:
# 2️⃣ 리디렉션 기반 조건 생성
df['is_short_url'] = df['shortened'] == 1
df['has_meta_refresh'] = df['meta_refresh_url'].notnull()

# 3️⃣ 두 조건 중 하나라도 해당되는 경우
df['is_redirection_suspect'] = df['is_short_url'] | df['has_meta_refresh']

# 4️⃣ 모든 조건이 다 해당되는 경우 (현재는 2가지 조건만 사용)
df['is_both'] = df['is_short_url'] & df['has_meta_refresh']

# 5️⃣ 개수 출력
print("\n🔍 조건 기반 요약:")
print(f"✔️ 단축 URL (shortened): {df['is_short_url'].sum()}개")
print(f"✔️ meta refresh 존재: {df['has_meta_refresh'].sum()}개")
print(f"✔️ 둘 다 해당: {df['is_both'].sum()}개")
print(f"✔️ 둘 중 하나라도 해당 (의심 URL): {df['is_redirection_suspect'].sum()}개")

# 6️⃣ 저장 (선택사항)
df.to_csv("../data/redirection_flagged.csv", index=False)



🔍 조건 기반 요약:
✔️ 단축 URL (shortened): 168개
✔️ meta refresh 존재: 7639개
✔️ 둘 다 해당: 1개
✔️ 둘 중 하나라도 해당 (의심 URL): 7806개


In [ ]:
# 6️⃣ 저장 (선택사항)
df.to_csv("../data/redirection_flagged.csv", index=False)